In [1]:
using LowLevelFEM, LinearAlgebra, SparseArrays
using Tensors
using StaticArrays

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
[ Info: Precompiling LowLevelFEMTensorsExt [142a104b-3cdc-540a-b57f-68e70732214f] (cache misses: wrong dep version loaded (4))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [3]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2, ν=0.48)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000604314s, CPU 0.000606s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000267947s, CPU 0.000267s)
Info    : Meshing 3D

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.48, 1621.6216216216199, 67.56756756756756, 1666.6666666666652, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)

load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * 30, fz=(x, y, z) -> (y - 0.5) * 30)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, var"#3#4"(), var"#5#6"(), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
f_ext = loadVector(prob, [load])

showDoFResults(f_ext, name="f")

1

In [6]:
μ = mat.μ
λ = mat.λ
κ = mat.κ

p = (μ=μ, λ=λ, κ=κ)

(μ = 67.56756756756756, λ = 1621.6216216216199, κ = 1666.6666666666652)

In [7]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])
I = unitTensor(F)

C = F' * F

elementwise TensorField
[[1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;]  …  [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;], [1.0; 0.0; … ; 0.0; 1.0;;]]

In [9]:
function ψ_StVenantKirchhoff(C, p)
    λ = p.λ
    μ = p.μ

    I = [1.0 0 0; 0 1 0; 0 0 1]

    E = (C - I) / 2

    return (λ * tr(E)^2 + 2μ * dot(E, E)) / 2
end


ψ_StVenantKirchhoff (generic function with 1 method)

In [10]:
function ψ_NeoHooke(C, p)
    μ = p.μ
    κ = p.κ

    I = [1.0 0 0; 0 1 0; 0 0 1]
    C_I = tr(C)
    C_III = det(C)
    J = √C_III

    return μ / 2 * (C_I / (C_III^(1 / 3)) - 3) + κ / 2 * log(J)^2
end


ψ_NeoHooke (generic function with 1 method)

In [11]:
ψ = ψ_StVenantKirchhoff
#ψ = ψ_NeoHooke

ψ_StVenantKirchhoff (generic function with 1 method)

In [12]:
S = IIPiolaKirchhoff(ψ, C, p)

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [13]:
P = F * S

elementwise TensorField
[[0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;]  …  [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 0.0;;]]

## Innen indul az iteráció

In [14]:
n = 30
uu = VectorField[]
for i in 1:n
    sc = i < 6 ? 5i : 30
    load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * sc, fz=(x, y, z) -> (y - 0.5) * sc)

    Kmat = materialTangentMatrix(prob, F=F, energy=ψ, params=p)
    Kgeo = initialStressMatrix(prob, energy=ψ, params=p, C=C)
    Kint = Kmat + Kgeo
    Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    Kext = externalTangentFollower(prob, [load], F=Fright)
    K = Kint - Kext

    f_int = internalForceVector(prob, energy=ψ, params=p, F=F)
    FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    f_ext = loadVector(prob, [load], F=FF)

    Δu = solveField(
        K,
        f_ext - f_int;
        support=[suppX]
    )

    fixed = constrainedDoFs(prob, [suppX])
    free = freeDoFs(prob, [suppX])
    #norm(Δu.a[:, 1])
    r = f_ext - f_int
    println("======== ", i, " ========")
    println("‖r_free‖  = ", norm(r.a[free, 1]))
    println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
    println("‖r‖ = ", norm(r.a[:, 1]))
    println("‖Δu‖ = ", norm(Δu.a[:, 1]))

    u += Δu
    push!(uu, u)
    F = I + u ∘ ∇
    C = F' * F
    S = IIPiolaKirchhoff(ψ, C, p)
    P = F * S

    if norm(Δu.a[:, 1]) < 0.01
        break
    end

end

======== 1 ========
‖r_free‖  = 0.12287947087587522
‖r_fixed‖ = 9.204143066617738e-14
‖r‖ = 0.12287947087587522
‖Δu‖ = 17.97729508796558
======== 2 ========
‖r_free‖  = 844.3718018430819
‖r_fixed‖ = 0.5402644873400225
‖r‖ = 844.3719746850016
‖Δu‖ = 15.56433044393727
======== 3 ========
‖r_free‖  = 128.6674235710958
‖r_fixed‖ = 0.5301839225478971
‖r‖ = 128.66851589808405
‖Δu‖ = 8.947335260091807
======== 4 ========
‖r_free‖  = 10.987587655093023
‖r_fixed‖ = 0.5164796788115847
‖r‖ = 10.9997197117462
‖Δu‖ = 6.472644808387345
======== 5 ========
‖r_free‖  = 6.664435790142124
‖r_fixed‖ = 0.49380219697683675
‖r‖ = 6.6827049172222495
‖Δu‖ = 1.0419405949370744
======== 6 ========
‖r_free‖  = 3.176506516550031
‖r_fixed‖ = 0.48505014089350373
‖r‖ = 3.213326514511966
‖Δu‖ = 2.8247743509662446
======== 7 ========
‖r_free‖  = 17.01467245313341
‖r_fixed‖ = 0.560344107868258
‖r‖ = 17.02389685726037
‖Δu‖ = 0.8226409173371321
======== 8 ========
‖r_free‖  = 1.5320624725105547
‖r_fixed‖ = 0.559446777381

In [15]:
u = mergeFields(uu)

nodal VectorField
[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.3625643724438271 -0.37400079737347464 … -0.9475279973807583 -0.8691607158779137; 0.3617310642991205 0.25020623320626895 … -0.8181268480363046 -0.9238022807083585]

In [16]:
showDoFResults(u, name="u_iter")

2

In [28]:
I = ScalarField(prob, "cube", 1.0, steps=30)
O = ScalarField(prob, "cube", 0.0, steps=30)
I = TensorField([I O O; O I O; O O I])
F = I + u ∘ ∇
C = F' * F
S = IIPiolaKirchhoff(ψ, C, p)

nodal TensorField
[2.227643411038371 1.4958005510594268 … 14.507346336173043 14.917211441922767; -2.4339697767877526 -2.406670827992887 … -8.344477983530087 -8.497887949834254; … ; -0.055098414499572854 -0.035988664869556125 … -0.250055671477974 -0.34053713335163066; 1126.1622508155954 220.97117266859044 … -1.0941891641747274 71.53067113738966]

In [29]:
showDoFResults(S, name="II P-K")

3

In [30]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [18]:
Kgeo.A

LoadError: UndefVarError: `Kgeo` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
#gmsh.finalize()